# Counting the number of points in each isochrone

In [ ]:
# import libraries
import geopandas as gpd
import os
import pandas as pd

In [ ]:
# Define the folder paths
isochrone_folder = "C:/Users/kbons/accessibility/Orleans/car_a_insee"
points_file = "C:/Users/kbons/accessibility/project_report/BPE_Orleans domains/transport_and_travel.csv"

In [ ]:
# Specify the isochrone values of interest
target_isochrones = [600, 1200, 1800, 2400, 3000, 3600]

In [ ]:
# Define values for the new fields
domain = "Transport and Travel"  # Replace with the desired domain 
mode = "Car"      # Replace with the desired mode 

In [ ]:
# Load the points shapefile once
points = pd.read_csv(points_file)

In [ ]:
# convert the points to a geodataframe
points = gpd.GeoDataFrame(
    points, 
    geometry=gpd.points_from_xy(points['LONGITUDE'], points['LATITUDE']),
    crs="EPSG:4326" 
)

In [ ]:
# Initialize a DataFrame to store results
all_results = []

In [ ]:
# Loop through each shapefile in the isochrone folder
for filename in os.listdir(isochrone_folder):
    if filename.endswith(".shp"):
        isochrone_path = os.path.join(isochrone_folder, filename)

        try:
            # Load the isochrone shapefile
            isochrones = gpd.read_file(isochrone_path)

            # Ensure the isochrones have a CRS defined; set a default if missing
            if isochrones.crs is None:
                isochrones = isochrones.set_crs("EPSG:4326")  # Replace "EPSG:4326" with the correct CRS if known

            # Ensure CRS matches
            if isochrones.crs != points.crs:
                isochrones = isochrones.to_crs(points.crs)

            # Filter isochrones to only keep those with the target values
            filtered_isochrones = isochrones[isochrones['time'].isin(target_isochrones)]

            # Skip if no relevant isochrones are found in this file
            if filtered_isochrones.empty:
                continue

            # Perform spatial join
            points_within_isochrones = gpd.sjoin(points, filtered_isochrones, how="inner", predicate="within")

            # Modify the filename to remove the first two and last four characters
            adjusted_filename = filename[2:-4]

            # Add columns for domain, mode, and adjusted filename
            points_within_isochrones["isochrone_file"] = adjusted_filename
            points_within_isochrones["domain"] = domain
            points_within_isochrones["mode"] = mode

            # select columns from "points_within_isochrones" and store in a new DataFrame
            results = points_within_isochrones[[
                "isochrone_file", "mode", "domain", "time", "LATITUDE", "LONGITUDE"
            ]]

            # Group by time to get counts of points
            point_counts = results.groupby(
                ["isochrone_file", "mode", "domain", "time"]
            ).size().reset_index(name="point_count")

            # Merge back with results to associate counts with each point row
            final_data = pd.merge(
                results, point_counts, on=["isochrone_file", "mode", "domain", "time"]
            )

            # Append to the main result list
            all_results.append(final_data)

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue



In [ ]:
# Loop through each shapefile in the isochrone folder
for filename in os.listdir(isochrone_folder):
    if filename.endswith(".shp"):
        isochrone_path = os.path.join(isochrone_folder, filename)

        try:
            # Load the isochrone shapefile
            isochrones = gpd.read_file(isochrone_path)

            # Ensure the isochrones have a CRS defined; set a default if missing
            if isochrones.crs is None:
                isochrones = isochrones.set_crs("EPSG:4326")  # Replace "EPSG:4326" with the correct CRS if known

            # Ensure CRS matches
            if isochrones.crs != points.crs:
                isochrones = isochrones.to_crs(points.crs)

            # Filter isochrones to only keep those with the target values
            # Adjust the attribute name (e.g., "value") to match your shapefile's column name
            filtered_isochrones = isochrones[isochrones['time'].isin(target_isochrones)]

            # Skip if no relevant isochrones are found in this file
            if filtered_isochrones.empty:
                continue

            # Perform spatial join
            points_within_isochrones = gpd.sjoin(points, filtered_isochrones, how="inner", predicate="within")

            # Group the results and count the points per isochrone
            point_counts = points_within_isochrones.groupby("time").size().reset_index(name="point_count")
            
            # Modify the filename to remove the first two and last four characters
            adjusted_filename = filename[2:-4]
            
            # Add the adjusted filename, domain, and mode to the DataFrame to the results for context
            point_counts["isochrone_file"] = adjusted_filename
            point_counts["domain"] = domain
            point_counts["mode"] = mode
           



            
            # Reorder the columns
            #point_counts = point_counts[["isochrone_file", "mode", "domain", "time", "point_count"]]

            # Append to the main result list
            all_results.append(point_counts)

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

In [ ]:
# Combine all results into a single DataFrame
if all_results:
    final_results = pd.concat(all_results, ignore_index=True)

    # Save to a CSV file
    final_results.to_csv("C:/Users/kbons/accessibility/project_report/points_in_isochrone_Orleans/car_transport_and_travel_ed.csv", index=False)

    print("Processing complete. Results saved to output.csv")
else:
    print("No matching isochrones found or no valid data processed.")